In [1]:
import random
import numpy as np
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import torch

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from peft import PeftModel, PeftConfig

# Load config first
peft_model_path = "./saved_model/gpt2-medium-lora-shakespeare/"
config = PeftConfig.from_pretrained(peft_model_path)

In [6]:
# Load base model and wrap with PEFT
base_model = GPT2LMHeadModel.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [8]:
# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(config.base_model_name_or_path)

In [9]:
def print_ot_for_prompt(input_):
    inputs = tokenizer(input_, return_tensors="pt").to(device)
    
    # Generate output (adjust parameters as needed)
    output_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=80,
        do_sample=True,
        top_k=40,
        temperature=0.80,
        repetition_penalty=1.15,
        pad_token_id=tokenizer.eos_token_id  # Important for GPT
    )
    
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    output_text = clean_repetition(output_text)
    print(output_text)
    print("----------------------------------------------------------------------")
    return output_text

def clean_repetition(output):
    import re
    # print("Input type:", type(output))  # Debug print
    output = str(output)
    output = re.sub(r'\b(nobel prize in the )\1', r'\1', output, flags=re.IGNORECASE)
    output = re.sub(r'\b(\w+)\s+\1\b', r'\1', output)
    output = re.sub(r'\b(for )\1', r'\1', output)
    return output

In [10]:
def print_model_stats(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total Parameters: {total_params / 1e6:.2f}M")

In [11]:
print_model_stats(model)

Total Parameters: 355.61M


In [12]:
model.eval()
print("model evaluation mode added")

model evaluation mode added


### Basic Prompts

In [13]:
prompt_lis = [
    "first citizen:",
    "hermione:"
]


for prompt in prompt_lis:
    print_ot_for_prompt(prompt)

first citizen:
To a father and to his son.
I am come from thee hence, sir, for thy sake; but I will not stay there long. But if thou wouldst leave me an hour, my brother, I must take care that thou wilt give some answer before he comes in at night. So bid him speak of the first day's affairs with me;
That
----------------------------------------------------------------------
hermione:
But, if thou be a good friend of mine,
I will hear thee well. I can assure thy pardon;--
Thou hast now heard me the word's truth and done all
for my safety that ever was spoken to me; but how
dost speak it? It is not in your nature.
And what! are thou willing to do no
----------------------------------------------------------------------


#### More prompts

In [14]:
print_ot_for_prompt("ACT I, SCENE I\nfirst citizen:")

ACT I, SCENE I
first citizen:
I have heard your words; so do you.
What is this?

First Citizen : O, no harm in that. To what!
So then we will leave town with all our provisions, my lord? and go up the river? which can
be done by horse or foot, till any of us shall meet again the
----------------------------------------------------------------------


'ACT I, SCENE I\nfirst citizen:\nI have heard your words; so do you.\nWhat is this?\n\nFirst Citizen : O, no harm in that. To what!\nSo then we will leave town with all our provisions, my lord? and go up the river? which can\nbe done by horse or foot, till any of us shall meet again the'

In [15]:
print_ot_for_prompt("to be or not to be ")

to be or not to be ?"
The king, as often as he thinks himself asked in his own mind, answers: "Now I find it my purpose.
I will go myself and look on the queen's son;
And see how good she is there!"--and again goes away with a great shout; 'tis very hard! for this time we must have one more.
----------------------------------------------------------------------


'to be or not to be ?"\nThe king, as often as he thinks himself asked in his own mind, answers: "Now I find it my purpose.\nI will go myself and look on the queen\'s son;\nAnd see how good she is there!"--and again goes away with a great shout; \'tis very hard! for this time we must have one more.'

### Utilizing a Hugging Face hosted model

In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("khotveer1/gpt2-medium-lora-finetuned-shakespeare")
model = AutoModelForCausalLM.from_pretrained("khotveer1/gpt2-medium-lora-finetuned-shakespeare")

prompt = "ACT I, SCENE I\nfirst citizen:"
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_length=80,
    do_sample=True,
    top_k=40,
    temperature=0.80,
    repetition_penalty=1.15,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ACT I, SCENE I
first citizen:
I hope he hath a true faith. O, then, let him make no reply; his eyes are full of hate
and grief, and the mind is filled with dread and anger.
O, now! how long will that last? He would not be my son-in law,
but one's own father: but 'tis
